In [1]:
import numpy as np
from susy_qm import calculate_Hamiltonian
import pennylane as qml
from pennylane import numpy as pnp
from qiskit.quantum_info import SparsePauliOp
from scipy.optimize import minimize, differential_evolution
import pandas as pd

In [227]:
cutoff = 4
potential = 'AHO'

In [228]:
#calculate Hamiltonian and expected eigenvalues
H = calculate_Hamiltonian(cutoff, potential)
eigenvalues, eigenvectors = np.linalg.eig(H)

min_index = np.argmin(eigenvalues)
min_eigenvalue = eigenvalues[min_index]
min_eigenvector = eigenvectors[:, min_index]

In [229]:
min_eigenvector

array([-0.        -0.j, -0.        -0.j, -0.        -0.j, -0.        -0.j,
        0.96367825+0.j, -0.        -0.j, -0.26706596-0.j, -0.        -0.j])

In [230]:
min_eigenvalue

np.complex128(-0.16478526068502214+0j)

In [231]:
#create qiskit Hamiltonian Pauli string
hamiltonian = SparsePauliOp.from_operator(H)
num_qubits = hamiltonian.num_qubits

In [232]:
dev = qml.device('default.qubit', wires=num_qubits, shots=10000)
@qml.qnode(dev)
def ansatz(params):
    
    qml.RY(params[0], wires=[0])
    qml.RY(params[1], wires=[num_qubits-2])
    
    return qml.probs()#qml.state() 

In [233]:
def cost_function(params):

    params = pnp.tensor(params, requires_grad=True)
    ansatz_state = ansatz(params)

    overlap = np.vdot(min_eigenvector, ansatz_state)
    overlap_squared = np.abs(overlap)**2  

    return (1 - overlap_squared)

In [247]:
def cost_function_hellinger(params):

    params = pnp.tensor(params, requires_grad=True)
    ansatz_state = ansatz(params)

    min_eigenvector_prob = np.abs(min_eigenvector)**2
    #ansatz_prob = np.abs(ansatz_state)**2
    ansatz_prob = ansatz_state

    hellinger_fidelity = np.sum(np.sqrt(np.outer(min_eigenvector_prob.real,ansatz_prob.real)))**2

    return (1 - hellinger_fidelity)


In [248]:
#x0 = np.random.random(params_shape).flatten()
x0 = np.random.rand(2)*2*np.pi
'''
res = minimize(
    cost_function,
    x0,
    method= "COBYLA",
    options= {'maxiter':10000}
)
'''
hres = minimize(
    cost_function_hellinger,
    x0,
    method= "COBYLA",
    options= {'maxiter':10000}
)

print(hres)

 message: Optimization terminated successfully.
 success: True
  status: 1
     fun: -5.056494030566714
       x: [ 1.581e+00  7.888e+00]
    nfev: 33
   maxcv: 0.0


In [249]:
bounds = [(0, 2 * np.pi) for _ in range(2)]
res = differential_evolution(cost_function_hellinger,
                                    bounds,
                                    maxiter=10000,
                                    tol=1e-8,
                                    atol=1e-8,
                                    strategy='randtobest1bin',
                                    popsize=20
                                    )

res

KeyboardInterrupt: 

In [245]:
dev = qml.device('default.qubit', wires=num_qubits, shots=None)
@qml.qnode(dev)
def circuit(params):
    
    qml.RY(params[0], wires=[0])
    qml.RY(params[1], wires=[num_qubits-2])
    
    return qml.expval(qml.Hermitian(H, wires=range(num_qubits))) 

In [246]:
circuit(res.x)

np.float64(0.8705798402842357)

In [ ]:
[ 4.486e+00  4.930e+00]
[ 3.149e+00  5.731e+00]

In [ ]:
data = []

for potential in ['QHO', 'AHO', 'DW']:
    for cutoff in [2, 4, 8, 16, 32, 64]:

        #calculate Hamiltonian and expected eigenvalues
        H = calculate_Hamiltonian(cutoff, potential)
        eigenvalues, eigenvectors = np.linalg.eig(H)

        min_index = np.argmin(eigenvalues)
        min_eigenvalue = eigenvalues[min_index]
        min_eigenvector = eigenvectors[:, min_index]

        #create qiskit Hamiltonian Pauli string
        hamiltonian = SparsePauliOp.from_operator(H)
        num_qubits = hamiltonian.num_qubits

        num_layers = 1
        params_shape = qml.StronglyEntanglingLayers.shape(n_layers=num_layers, n_wires=num_qubits)
        x0 = np.random.random(params_shape).flatten()

        # Initialize device
        dev = qml.device("default.qubit", wires=num_qubits)

        # Define the parameterized ansatz
        @qml.qnode(dev)
        def ansatz(params):
            qml.StronglyEntanglingLayers(params, wires=range(num_qubits), imprimitive=qml.CZ)
            return qml.state()  # Return the statevector
        
        def cost_function(params):

            params = pnp.tensor(params.reshape(params_shape), requires_grad=True)
            ansatz_state = ansatz(params)

            overlap = np.vdot(min_eigenvector, ansatz_state)
            overlap_squared = np.abs(overlap)**2  

            return (1 - overlap_squared)
        
        def cost_function_hellinger(params):

            params = pnp.tensor(params.reshape(params_shape), requires_grad=True)
            ansatz_state = ansatz(params)

            min_eigenvector_prob = np.abs(min_eigenvector)**2
            ansatz_prob = np.abs(ansatz_state)**2

            hellinger_fidelity = np.sum(np.sqrt(min_eigenvector_prob * ansatz_prob))

            return (1 - hellinger_fidelity)
        

        overlap_res = minimize(
            cost_function,
            x0,
            method= "COBYLA",
            options= {'maxiter':10000, 'tol': 1e-8}
        )

        hellinger_res = minimize(
            cost_function_hellinger,
            x0,
            method= "COBYLA",
            options= {'maxiter':10000, 'tol': 1e-8}
        )

        row_data = {'potential': potential, 'cutoff': cutoff, 'overlap': overlap_res.fun, 'hellinger': hellinger_res.fun}

        data.append(row_data)     

In [ ]:
pd.DataFrame(data)

## Real Amplitudes

In [ ]:
cutoff = 128
potential = 'AHO'

In [ ]:
#calculate Hamiltonian and expected eigenvalues
H = calculate_Hamiltonian(cutoff, potential)
eigenvalues, eigenvectors = np.linalg.eig(H)

min_index = np.argmin(eigenvalues)
min_eigenvalue = eigenvalues[min_index]
min_eigenvector = eigenvectors[:, min_index]

In [ ]:
#create qiskit Hamiltonian Pauli string
hamiltonian = SparsePauliOp.from_operator(H)
num_qubits = hamiltonian.num_qubits

In [ ]:
# Parameters
repetition = 1

# Initialize device
dev = qml.device("default.qubit", wires=num_qubits)
    
# Define the ansatz circuit
@qml.qnode(dev)
def real_amplitudes(params, repetition, num_qubits):
    param_index = 0
    for _ in range(repetition):

        # Apply RY rotations
        for i in range(num_qubits):
            qml.RY(params[param_index], wires=i)
            param_index += 1

        # Apply entanglement
        for j in reversed(range(1, num_qubits)):  # Reverse linear entanglement
            qml.CNOT(wires=[j - 1, j])

        # Apply RY rotations
        for k in range(num_qubits):
            qml.RY(params[param_index], wires=k)
            param_index += 1

    return qml.state()

# Number of parameters required for this ansatz
n_params = repetition * 2*num_qubits
params = np.random.uniform(0, 2 * np.pi, size=n_params)


In [ ]:
@qml.qnode(dev)
def circuit(params):
    param_index=0
    for i in range(0, num_qubits-2):
        qml.RY(params[param_index], wires=i)
        param_index += 1

    # Apply entanglement
    for j in reversed(range(1, num_qubits-2)):
        qml.CNOT(wires=[j - 1, j])

    # Apply RY rotations
    for k in range(0, num_qubits-2):
        qml.RY(params[param_index], wires=k)
        param_index += 1
    
    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))     

In [ ]:
n_params = repetition * 2*num_qubits
params = np.random.uniform(0, 2 * np.pi, size=n_params)
drawer = qml.draw(circuit)
print(drawer(params))

In [ ]:
drawer = qml.draw(real_amplitudes)
print(drawer(params, repetition, num_qubits))

In [ ]:
def cost_function(params):

    ansatz_state = real_amplitudes(params, repetition, num_qubits)

    overlap = np.vdot(min_eigenvector, ansatz_state)
    overlap_squared = np.abs(overlap)**2  

    return (1 - overlap_squared)

In [ ]:
def cost_function_hellinger(params):

    ansatz_state = real_amplitudes(params, repetition, num_qubits)

    min_eigenvector_prob = np.abs(min_eigenvector)**2
    ansatz_prob = np.abs(ansatz_state)**2

    hellinger_fidelity = np.sum(np.sqrt(min_eigenvector_prob * ansatz_prob))

    return (1 - hellinger_fidelity)


In [ ]:
x0 = np.random.uniform(0, 2 * np.pi, size=n_params)

res = minimize(
    cost_function,
    x0,
    method= "COBYLA",
    options= {'maxiter':10000, 'tol': 1e-8}
)


In [ ]:
data = []

repetition = 1

for potential in ['QHO', 'AHO', 'DW']:
    for cutoff in [2, 4, 8, 16, 32, 64]:

        #calculate Hamiltonian and expected eigenvalues
        H = calculate_Hamiltonian(cutoff, potential)
        eigenvalues, eigenvectors = np.linalg.eig(H)

        min_index = np.argmin(eigenvalues)
        min_eigenvalue = eigenvalues[min_index]
        min_eigenvector = eigenvectors[:, min_index]

        #create qiskit Hamiltonian Pauli string
        hamiltonian = SparsePauliOp.from_operator(H)
        num_qubits = hamiltonian.num_qubits
        n_params = repetition * 2*num_qubits

        x0 = np.random.uniform(0, 2 * np.pi, size=n_params)

        # Initialize device
        dev = qml.device("default.qubit", wires=num_qubits)

        # Define the ansatz circuit
        @qml.qnode(dev)
        def real_amplitudes(params, repetition, num_qubits):
            param_index = 0
            for _ in range(repetition):

                # Apply RY rotations
                for i in range(num_qubits):
                    qml.RY(params[param_index], wires=i)
                    param_index += 1

                # Apply entanglement
                for j in reversed(range(1, num_qubits)):  # Reverse linear entanglement
                    qml.CNOT(wires=[j - 1, j])

                # Apply RY rotations
                for k in range(num_qubits):
                    qml.RY(params[param_index], wires=k)
                    param_index += 1

            return qml.state()

        overlap_res = minimize(
            cost_function,
            x0,
            method= "COBYLA",
            options= {'maxiter':10000, 'tol': 1e-8}
        )

        hellinger_res = minimize(
            cost_function_hellinger,
            x0,
            method= "COBYLA",
            options= {'maxiter':10000, 'tol': 1e-8}
        )

        row_data = {'potential': potential, 'cutoff': cutoff, 'overlap': overlap_res.fun, 'hellinger': hellinger_res.fun}

        data.append(row_data)     

In [ ]:
pd.DataFrame(data)

In [ ]:
data = []

repetition = 1

#variables
max_iter = 10000
strategy = "randtobest1bin"
tol = 1e-3
abs_tol = 1e-3
popsize = 20

for potential in ['QHO', 'AHO', 'DW']:
    for cutoff in [2]:#, 4, 8, 16, 32, 64]:

        #calculate Hamiltonian and expected eigenvalues
        H = calculate_Hamiltonian(cutoff, potential)
        eigenvalues, eigenvectors = np.linalg.eig(H)

        min_index = np.argmin(eigenvalues)
        min_eigenvalue = eigenvalues[min_index]
        min_eigenvector = eigenvectors[:, min_index]

        #create qiskit Hamiltonian Pauli string
        hamiltonian = SparsePauliOp.from_operator(H)
        num_qubits = hamiltonian.num_qubits
        n_params = repetition * 2*num_qubits

        bounds = [(0, 2 * np.pi) for _ in range(n_params)]
        # Initialize device
        dev = qml.device("default.qubit", wires=num_qubits)

        # Define the ansatz circuit
        @qml.qnode(dev)
        def real_amplitudes(params, repetition, num_qubits):
            param_index = 0
            for _ in range(repetition):

                # Apply RY rotations
                for i in range(num_qubits):
                    qml.RY(params[param_index], wires=i)
                    param_index += 1

                # Apply entanglement
                for j in reversed(range(1, num_qubits)):  # Reverse linear entanglement
                    qml.CNOT(wires=[j - 1, j])

                # Apply RY rotations
                for k in range(num_qubits):
                    qml.RY(params[param_index], wires=k)
                    param_index += 1

            return qml.state()

        overlap_res = differential_evolution(
            cost_function,
            bounds,
            maxiter=max_iter,
            tol=tol,
            atol=abs_tol,
            strategy=strategy,
            popsize=popsize
        )

        hellinger_res = differential_evolution(
            cost_function_hellinger,
            bounds,
            maxiter=max_iter,
            tol=tol,
            atol=abs_tol,
            strategy=strategy,
            popsize=popsize
        )

        row_data = {'potential': potential, 'cutoff': cutoff, 'overlap': overlap_res.fun, 'hellinger': hellinger_res.fun}

        data.append(row_data)     

In [ ]:
pd.DataFrame(data)